In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.linear_model import  LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
import optuna

In [4]:
import joblib
import os

# Tratamento dos dados

In [5]:
spotify_youtube_pt1 = pd.read_csv("Dados/spotify_youtube_up_pt1.csv")
spotify_youtube_pt2 = pd.read_csv("Dados/spotify_youtube_up_pt2.csv")

In [6]:
data_extracao = pd.to_datetime("2023-02-07")
spotify_youtube_pt1["upload_date"] = pd.to_datetime(spotify_youtube_pt1["upload_date"])
spotify_youtube_pt1["dias_na_plataforma"] = (data_extracao - spotify_youtube_pt1["upload_date"]).dt.days

spotify_youtube_pt2["upload_date"] = pd.to_datetime(spotify_youtube_pt2["upload_date"])
spotify_youtube_pt2["dias_na_plataforma"] = (data_extracao - spotify_youtube_pt2["upload_date"]).dt.days

In [7]:
spotify_youtube = pd.concat([spotify_youtube_pt1,spotify_youtube_pt2])

In [8]:
spotify_youtube_dummies= pd.get_dummies(spotify_youtube,columns=['Album_type'], dtype=int)

In [9]:
spotify_youtube_dummies['artist_number'] = spotify_youtube_dummies['Artist'].str.split(',').str.len()
spotify_youtube_dummies['engagement_rate'] = spotify_youtube_dummies.apply(lambda row: (row['Likes'] + row['Comments']) / row['Views'] * 100 if row['Views'] > 0 else 0, axis=1)

In [10]:
columns_drop=['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Uri','Url_youtube', 'Title', 'Channel', 'Views', 'Likes','Comments', 'Description', 'Licensed', 'official_video', 'upload_date']
spotify_youtube_df=spotify_youtube_dummies.drop(columns=columns_drop)

In [11]:
spotify_youtube_limpo=spotify_youtube_df.dropna(subset=['Danceability','Energy','Key','Loudness','Speechiness','Acousticness', 'Instrumentalness','Liveness','Valence','Tempo','Duration_ms','Stream','engagement_rate'])

# Preparação dos dados para os experimentos

In [12]:
youtube_30_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<31]
youtube_90_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<91]
youtube_365_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<366]

In [13]:
spotify_youtube_df_366_more = spotify_youtube_limpo.drop(youtube_365_df.index)
spotify_youtube_df_91_365 = youtube_365_df.drop(youtube_90_df.index)
spotify_youtube_df_31_90 = youtube_365_df.drop(youtube_30_df.index)

In [14]:
youtube_30_df=youtube_30_df[youtube_30_df['engagement_rate']<9.338507725109846]
spotify_youtube_df_31_90=spotify_youtube_df_31_90[spotify_youtube_df_31_90['engagement_rate']<4.602401258613792]
spotify_youtube_df_91_365=spotify_youtube_df_91_365[spotify_youtube_df_91_365['engagement_rate']<4.253191569584401]
spotify_youtube_df_366_more=spotify_youtube_df_366_more[spotify_youtube_df_366_more['engagement_rate']<2.251193229196198]

In [15]:
youtube_30_df_x = youtube_30_df.drop('engagement_rate',axis=1)
youtube_30_df_y = youtube_30_df['engagement_rate']

spotify_youtube_df_31_90_x = spotify_youtube_df_31_90.drop('engagement_rate',axis=1)
spotify_youtube_df_31_90_y = spotify_youtube_df_31_90['engagement_rate']

spotify_youtube_df_91_365_x = spotify_youtube_df_91_365.drop('engagement_rate',axis=1)
spotify_youtube_df_91_365_y = spotify_youtube_df_91_365['engagement_rate']

spotify_youtube_df_366_more_x = spotify_youtube_df_366_more.drop('engagement_rate',axis=1)
spotify_youtube_df_366_more_y = spotify_youtube_df_366_more['engagement_rate']

In [ ]:
spotify_youtube_30X_train_val, spotify_youtube_30X_test, spotify_youtube_30y_train_val, spotify_youtube_30y_test = train_test_split(youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=42)
spotify_youtube_30X_train, spotify_youtube_30X_val, spotify_youtube_30y_train, spotify_youtube_30y_val = train_test_split(spotify_youtube_30X_train_val, spotify_youtube_30y_train_val, test_size=0.20, random_state=42)

In [17]:
#spotify_youtube_30X_train_val, spotify_youtube_30X_test, spotify_youtube_30y_train_val, spotify_youtube_30y_test = train_test_split(youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=42)
spotify_youtube_df_31_90X_train, spotify_youtube_df_31_90X_test, spotify_youtube_df_31_90y_train, spotify_youtube_df_31_90y_test = train_test_split(spotify_youtube_df_31_90_x, spotify_youtube_df_31_90_y, test_size=0.25, random_state=42)
spotify_youtube_df_91_365X_train, spotify_youtube_df_91_365X_test, spotify_youtube_df_91_365y_train, spotify_youtube_df_91_365y_test = train_test_split(spotify_youtube_df_91_365_x, spotify_youtube_df_91_365_y, test_size=0.25, random_state=42)
#spotify_youtube_30X_train, spotify_youtube_30X_val, spotify_youtube_30y_train, spotify_youtube_30y_val = train_test_split(spotify_youtube_30X_train_val, spotify_youtube_30y_train_val, test_size=0.20, random_state=42)

In [17]:
# Dados de treino 
sy_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train,spotify_youtube_df_91_365X_train,spotify_youtube_df_366_more_x])
sy_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train,spotify_youtube_df_91_365y_train,spotify_youtube_df_366_more_y])

# Normalização do treino
scaler_all = MinMaxScaler()
sy_x_train_scaled = scaler_all.fit_transform(sy_x_train)

In [18]:
# Normalização do treino de 30 dias
sy_30_x_train_scaled = scaler_all.transform(spotify_youtube_30X_train)

In [19]:
# Dados para treinar o modelo com dados de 0 a 90 dias
sy_90_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train])
sy_90_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train])

# Normalização do treino de 90 dias
sy_90_x_train_scaled = scaler_all.transform(sy_90_x_train)

In [20]:
# Dados para treinar o modelo com dados de 0 a 365 dias
sy_365_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train,spotify_youtube_df_91_365X_train])
sy_365_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train,spotify_youtube_df_91_365y_train])

# Normalização do treino de 90 dias
sy_365_x_train_scaled = scaler_all.transform(sy_365_x_train)

In [ ]:
# Para testar o desempenho do modelo com videos com ate 30 dias
sy_30_x_test=spotify_youtube_30X_test
sy_30_y_test=spotify_youtube_30y_test

# validação de 30 dias
sy_30_x_val=spotify_youtube_30X_val
sy_30_y_val=spotify_youtube_30y_val

# Para testar o desempenho do modelo com videos com ate 90 dias
sy_90_x_test=pd.concat([spotify_youtube_30X_test,spotify_youtube_df_31_90X_test])
sy_90_y_test=pd.concat([spotify_youtube_30y_test,spotify_youtube_df_31_90y_test])

# Para testar o desempenho do modelo com videos com ate 1 ano
sy_365_x_test=pd.concat([spotify_youtube_30X_test,spotify_youtube_df_31_90X_test,spotify_youtube_df_91_365X_test])
sy_365_y_test=pd.concat([spotify_youtube_30y_test,spotify_youtube_df_31_90y_test,spotify_youtube_df_91_365y_test])

In [22]:
# Normalizção dos dados de teste de 30 dias
sy_30_x_test_scaled = scaler_all.transform(sy_30_x_test)

# Normalizção dos dados de teste de 90 dias
sy_90_x_test_scaled = scaler_all.transform(sy_90_x_test)

# Normalizção dos dados de teste de 365 dias
sy_365_x_test_scaled = scaler_all.transform(sy_365_x_test)

# Normalização dos dados de validação de 30 dias
sy_30_x_val_scaled = scaler_all.transform(sy_30_x_val)

# Tuning dos modelos

## Random Forest

Resultados:

GridSearch:
* Média dos MSEs: 3.4293
* Desvio padrão dos MSEs: 0.4456
* Melhor MSE individual: 2.5248
* Tempo: 82m 17s

Optuna:


Fazendo o tuning com GridSearch

In [ ]:
# Definindo os hiperparâmetros para o GridSearch
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_samples': [0.5, 0.75],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [30, 40, 50],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]
}

# Inicializando o modelo Random Forest
rf = RandomForestRegressor(random_state=42)

# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Configurando o GridSearchCV com PredefinedSplit
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=ps,  # Passa o PredefinedSplit
    scoring='neg_mean_squared_error',
    verbose=0,
    n_jobs=1
)

# Treinando o GridSearchCV
grid_search.fit(X_full, y_full)

# Obtendo o melhor modelo
best_rf = grid_search.best_estimator_

# Avaliando o modelo nos dados de teste
sy_30_x_test_scaled_df = pd.DataFrame(
    sy_30_x_test_scaled,
    columns=sy_x_train.columns,
    index=sy_30_x_test.index
)
predictions = best_rf.predict(sy_30_x_test_scaled_df)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (mse) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_random_forest_model_grid.pkl')
joblib.dump(best_rf, model_path)
print(f"Modelo salvo em: {model_path}")

Mean Squared Error (mse) on test data: 2.9452172749232077
Modelo salvo em: Modelos\best_random_forest_model_grid.pkl


In [26]:
# Hiperparâmetros para o GridSearch
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_samples': [0.5, 0.75],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [30, 40, 50],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4]
}

mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")
    spotify_youtube_30X_train_val, spotify_youtube_30X_test, spotify_youtube_30y_train_val, spotify_youtube_30y_test = train_test_split(youtube_30_df_x, youtube_30_df_y, test_size=0.25,random_state=40+i)
    spotify_youtube_30X_train, spotify_youtube_30X_val, spotify_youtube_30y_train, spotify_youtube_30y_val = train_test_split(spotify_youtube_30X_train_val, spotify_youtube_30y_train_val, test_size=0.20,random_state=40+i)

    # Dados de treino
    sy_x_train = pd.concat([spotify_youtube_30X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([spotify_youtube_30y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler_all = MinMaxScaler()
    sy_x_train_scaled = scaler_all.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler_all.transform(spotify_youtube_30X_val)
    sy_30_x_test_scaled = scaler_all.transform(spotify_youtube_30X_test)

    sy_30_y_val = spotify_youtube_30y_val
    sy_30_y_test = spotify_youtube_30y_test

    # Reconstrução dos DataFrames normalizados
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=spotify_youtube_30X_val.columns, index=spotify_youtube_30X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=sy_x_train.columns, index=spotify_youtube_30X_test.index)

    # Junta treino e validação
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, sy_30_y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(spotify_youtube_30X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Inicializa o modelo base
    rf = RandomForestRegressor(random_state=i)

    # GridSearch
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=ps,
        scoring='neg_mean_squared_error',
        verbose=0,
        n_jobs=-1
    )

    grid_search.fit(X_full, y_full)

    # Recupera os melhores hiperparâmetros
    best_params = grid_search.best_params_

    # Reentreina o modelo com os melhores hiperparâmetros
    best_rf = RandomForestRegressor(**best_params, random_state=42)
    best_rf.fit(X_full, y_full)

    # Previsão no teste
    predictions = best_rf.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(sy_30_y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    # Atualiza o melhor modelo
    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = best_rf

# Cálculo da média e do desvio padrão
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_rf_model_grid.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")


Rodada 1/10
MSE da rodada 1: 3.4494

Rodada 2/10
MSE da rodada 2: 3.2273

Rodada 3/10
MSE da rodada 3: 2.9044

Rodada 4/10
MSE da rodada 4: 3.9731

Rodada 5/10
MSE da rodada 5: 2.5248

Rodada 6/10
MSE da rodada 6: 4.0273

Rodada 7/10
MSE da rodada 7: 3.7666

Rodada 8/10
MSE da rodada 8: 3.6979

Rodada 9/10
MSE da rodada 9: 3.3297

Rodada 10/10
MSE da rodada 10: 3.3925

Resumo após 10 execuções:
Média dos MSEs: 3.4293
Desvio padrão dos MSEs: 0.4456
Melhor MSE individual: 2.5248
Melhor modelo salvo em: Modelos\best_rf_model_grid.pkl


Fazendo o tuning com Optuna

In [35]:
# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Função objetivo para o Optuna
def objective(trial):
    # Sugerindo os hiperparâmetros
    n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
    max_depth = trial.suggest_int("max_depth", 10, 50, step=10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10, step=2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4, step=1)
    max_samples = trial.suggest_float("max_samples", 0.5, 1.0, step=0.05)
    max_features = trial.suggest_categorical("max_features", ['sqrt', 'log2'])


    
    

    # Criando o modelo com os hiperparâmetros sugeridos
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_samples=max_samples,
        max_features=max_features,
        
        
        random_state=42
    )

    # Treinando o modelo nos dados de treino
    rf.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])

    # Avaliando o modelo nos dados de validação
    predictions = rf.predict(X_full[ps.test_fold == 0])
    mse = mean_squared_error(y_full[ps.test_fold == 0], predictions)

    return mse  # O objetivo é minimizar o MSE

# Criando o estudo do Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Número de tentativas

# Obtendo os melhores hiperparâmetros
best_params = study.best_params
print("Melhores hiperparâmetros:", best_params)

# Treinando o modelo final com os melhores hiperparâmetros
best_rf = RandomForestRegressor(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    random_state=42
)
best_rf.fit(sy_x_train_scaled, sy_y_train)

# Avaliando o modelo nos dados de teste
predictions = best_rf.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_random_forest_model_optuna.pkl')
joblib.dump(best_rf, model_path)
print(f"Modelo salvo em: {model_path}")

[I 2025-04-21 18:41:19,427] A new study created in memory with name: no-name-a6adede2-6e97-44a2-b830-b9359b490f1d
[I 2025-04-21 18:41:29,856] Trial 0 finished with value: 3.6594923027500004 and parameters: {'n_estimators': 350, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_samples': 0.95, 'max_features': 'log2'}. Best is trial 0 with value: 3.6594923027500004.
[I 2025-04-21 18:41:37,267] Trial 1 finished with value: 3.8796388232432424 and parameters: {'n_estimators': 350, 'max_depth': 40, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_samples': 0.55, 'max_features': 'sqrt'}. Best is trial 0 with value: 3.6594923027500004.
[I 2025-04-21 18:41:44,639] Trial 2 finished with value: 3.7630106068662728 and parameters: {'n_estimators': 350, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_samples': 0.9, 'max_features': 'log2'}. Best is trial 0 with value: 3.6594923027500004.
[I 2025-04-21 18:41:51,941] Trial 3 finished with value: 3.87240710807

Melhores hiperparâmetros: {'n_estimators': 50, 'max_depth': 40, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_samples': 0.95, 'max_features': 'log2'}
Mean Squared Error (MSE) on test data: 2.951254082497033
Modelo salvo em: Modelos\best_random_forest_model_optuna.pkl


In [33]:
mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=40+i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=40+i
    )

    # Concatena dados externos com treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    # Reconstruindo DataFrames
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=X_test.columns, index=X_test.index)

    # Junta treino + val
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Função objetivo do Optuna
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int("n_estimators", 50, 500, step=50),
            'max_depth': trial.suggest_int("max_depth", 10, 50, step=10),
            'min_samples_split': trial.suggest_int("min_samples_split", 2, 10, step=2),
            'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 4),
            'max_samples': trial.suggest_float("max_samples", 0.5, 1.0, step=0.05),
            'max_features': trial.suggest_categorical("max_features", ['sqrt', 'log2']),
            'random_state': 42,
        }

        model = RandomForestRegressor(**params)
        model.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])
        preds = model.predict(X_full[ps.test_fold == 0])
        mse = mean_squared_error(y_full[ps.test_fold == 0], preds)
        return mse

    # Rodando o estudo
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, show_progress_bar=False)

    best_params = study.best_params
    print("Melhores hiperparâmetros:", best_params)

    # Treina modelo final com treino + val
    final_model = RandomForestRegressor(**best_params)
    final_model.fit(X_full, y_full)

    # Avaliação em teste
    predictions = final_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = final_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_random_forest_model_optuna.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")

[I 2025-05-04 18:59:24,948] A new study created in memory with name: no-name-838c1965-6b54-4d2c-868a-1854f7422032



Rodada 1/10


[I 2025-05-04 18:59:34,231] Trial 0 finished with value: 4.645826088171258 and parameters: {'n_estimators': 300, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_samples': 0.95, 'max_features': 'log2'}. Best is trial 0 with value: 4.645826088171258.
[I 2025-05-04 18:59:39,640] Trial 1 finished with value: 4.531609873723485 and parameters: {'n_estimators': 150, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': 0.95, 'max_features': 'log2'}. Best is trial 1 with value: 4.531609873723485.
[I 2025-05-04 18:59:47,010] Trial 2 finished with value: 4.941853549914649 and parameters: {'n_estimators': 300, 'max_depth': 30, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_samples': 0.75, 'max_features': 'log2'}. Best is trial 1 with value: 4.531609873723485.
[I 2025-05-04 18:59:48,110] Trial 3 finished with value: 4.742038092280257 and parameters: {'n_estimators': 50, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_samples':

Melhores hiperparâmetros: {'n_estimators': 350, 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': 0.95, 'max_features': 'log2'}


[I 2025-05-04 19:07:07,660] A new study created in memory with name: no-name-688c987f-2f8f-4771-93cc-79d3a8aa8f53


...

Melhores hiperparâmetros: {'n_estimators': 50, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_samples': 0.9, 'max_features': 'log2'}


[I 2025-05-04 20:05:33,240] A new study created in memory with name: no-name-0bfa4954-3c25-419b-9684-510311dd7c25


MSE da rodada 9: 3.3814

Rodada 10/10


[I 2025-05-04 20:05:39,858] Trial 0 finished with value: 3.3793367863372716 and parameters: {'n_estimators': 250, 'max_depth': 50, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_samples': 0.8, 'max_features': 'log2'}. Best is trial 0 with value: 3.3793367863372716.
[I 2025-05-04 20:05:50,890] Trial 1 finished with value: 3.505442649138936 and parameters: {'n_estimators': 450, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_samples': 0.8, 'max_features': 'log2'}. Best is trial 0 with value: 3.3793367863372716.
[I 2025-05-04 20:06:01,474] Trial 2 finished with value: 3.363251881947734 and parameters: {'n_estimators': 400, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_samples': 0.9, 'max_features': 'sqrt'}. Best is trial 2 with value: 3.363251881947734.
[I 2025-05-04 20:06:05,890] Trial 3 finished with value: 3.59673983346764 and parameters: {'n_estimators': 300, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_samples

Melhores hiperparâmetros: {'n_estimators': 300, 'max_depth': 40, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_samples': 1.0, 'max_features': 'sqrt'}
MSE da rodada 10: 3.3078

Resumo após 10 execuções:
Média dos MSEs: 3.3308
Desvio padrão dos MSEs: 0.3906
Melhor MSE individual: 2.6242
Melhor modelo salvo em: Modelos\best_random_forest_model_optuna.pkl


## Linear Regressor

Resultados:

GridSearch:
* Média dos MSEs: 7.5433
* Desvio padrão dos MSEs: 1.1953
* Melhor MSE individual: 5.7530
* Tempo: 22s

Optuna:
* Média dos MSEs: 7.5433
* Desvio padrão dos MSEs: 1.1953
* Melhor MSE individual: 5.7530
* Tempo: 0.3s


Fazendo tuning com GridSearch

In [47]:
# Definindo os hiperparâmetros para o GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializando o modelo Random Forest
lin_reg = LinearRegression() # Regressor logistico

# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Configurando o GridSearchCV com PredefinedSplit
grid_search = GridSearchCV(
    estimator=lin_reg,
    param_grid={},  # Sem hiperparâmetros para ajustar
    cv=ps,  # Passa o PredefinedSplit
    scoring='neg_mean_squared_error',
    verbose=3,
    n_jobs=-1,
)

# Treinando o GridSearchCV
grid_search.fit(X_full, y_full)

# Obtendo o melhor modelo
best_lin_reg = grid_search.best_estimator_

# Avaliando o modelo nos dados de teste
predictions = best_lin_reg.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (mse) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_Linear_Regressor_model_grid.pkl')
joblib.dump(best_lin_reg, model_path)
print(f"Modelo salvo em: {model_path}")

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Mean Squared Error (mse) on test data: 6.18329638505496
Modelo salvo em: Modelos\best_Linear_Regressor_model_grid.pkl


c:\Faculdade\TCC_Music\TCC_Music\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [31]:
mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=50+i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=50+i
    )

    # Concatena dados externos com treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    # Reconstruindo DataFrames
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=X_test.columns, index=X_test.index)

    # Junta treino + val
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Modelo
    lin_reg = LinearRegression()

    # GridSearch (sem parâmetros a otimizar nesse caso)
    grid_search = GridSearchCV(
        estimator=lin_reg,
        param_grid={},  # LinearRegression não tem hiperparâmetros relevantes aqui
        cv=ps,
        scoring='neg_mean_squared_error',
        verbose=0,
        n_jobs=-1,
    )
    grid_search.fit(X_full, y_full)
    best_model = grid_search.best_estimator_

    # Avaliação
    predictions = best_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = best_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_Linear_Regressor_model_grid.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")


Rodada 1/10
MSE da rodada 1: 8.2045

Rodada 2/10
MSE da rodada 2: 7.0143

Rodada 3/10
MSE da rodada 3: 8.7981

Rodada 4/10
MSE da rodada 4: 8.9483

Rodada 5/10
MSE da rodada 5: 8.6841

Rodada 6/10
MSE da rodada 6: 6.3068

Rodada 7/10
MSE da rodada 7: 5.8110

Rodada 8/10
MSE da rodada 8: 5.7530

Rodada 9/10
MSE da rodada 9: 7.3891

Rodada 10/10
MSE da rodada 10: 8.5238

Resumo após 10 execuções:
Média dos MSEs: 7.5433
Desvio padrão dos MSEs: 1.1953
Melhor MSE individual: 5.7530
Melhor modelo salvo em: Modelos\best_Linear_Regressor_model_grid.pkl


Fazendo o Tuning com Optuna

In [48]:
# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Função objetivo para o Optuna
def objective(trial):
    # Inicializando o modelo Linear Regression
    lin_reg = LinearRegression()

    # Treinando o modelo nos dados de treino
    lin_reg.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])

    # Avaliando o modelo nos dados de validação
    predictions = lin_reg.predict(X_full[ps.test_fold == 0])
    mse = mean_squared_error(y_full[ps.test_fold == 0], predictions)

    return mse  # O objetivo é minimizar o MSE

# Criando o estudo do Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)  # Apenas 1 tentativa, pois não há hiperparâmetros para ajustar

# Treinando o modelo final
best_lin_reg = LinearRegression()
best_lin_reg.fit(sy_x_train_scaled, sy_y_train)

# Avaliando o modelo nos dados de teste
predictions = best_lin_reg.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_linear_regressor_model_optuna.pkl')
joblib.dump(best_lin_reg, model_path)
print(f"Modelo salvo em: {model_path}")

[I 2025-04-17 15:47:13,647] A new study created in memory with name: no-name-8decee07-315f-4476-8cb2-7e5be21e71fa
[I 2025-04-17 15:47:13,671] Trial 0 finished with value: 7.789384479414533 and parameters: {}. Best is trial 0 with value: 7.789384479414533.


Mean Squared Error (MSE) on test data: 6.245837740756641
Modelo salvo em: Modelos\best_linear_regressor_model_optuna.pkl


In [32]:
mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados (treino/val/teste)
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=50 + i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=50 + i
    )

    # Concatena dados externos com treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    # Reconstruindo DataFrames normalizados
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=X_test.columns, index=X_test.index)

    # Junta treino + validação
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # PredefinedSplit: -1 (treino), 0 (validação)
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Função objetivo do Optuna (sem hiperparâmetros para ajustar)
    def objective(trial):
        model = LinearRegression()
        model.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])
        preds = model.predict(X_full[ps.test_fold == 0])
        mse = mean_squared_error(y_full[ps.test_fold == 0], preds)
        return mse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=1, show_progress_bar=False)

    # Treino final com treino + validação
    best_model = LinearRegression()
    best_model.fit(X_full, y_full)

    # Avaliação em teste
    predictions = best_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = best_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_linear_regressor_model_optuna.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")

[I 2025-05-04 18:56:13,803] A new study created in memory with name: no-name-f897356b-c3a3-4b9f-9013-271bd14871da
[I 2025-05-04 18:56:13,816] Trial 0 finished with value: 7.666537239804746 and parameters: {}. Best is trial 0 with value: 7.666537239804746.
[I 2025-05-04 18:56:13,830] A new study created in memory with name: no-name-4aa5a975-439a-4168-87d4-00b426c01ef3
[I 2025-05-04 18:56:13,849] Trial 0 finished with value: 6.972907251732231 and parameters: {}. Best is trial 0 with value: 6.972907251732231.
[I 2025-05-04 18:56:13,862] A new study created in memory with name: no-name-3d8ae118-3e62-4294-8929-2c1f23aa72f7
[I 2025-05-04 18:56:13,882] Trial 0 finished with value: 6.508262635018185 and parameters: {}. Best is trial 0 with value: 6.508262635018185.
[I 2025-05-04 18:56:13,903] A new study created in memory with name: no-name-b22f96da-40f1-4ffe-8a33-410faa93a5a6
[I 2025-05-04 18:56:13,912] Trial 0 finished with value: 6.850767159021821 and parameters: {}. Best is trial 0 with va


Rodada 1/10
MSE da rodada 1: 8.2045

Rodada 2/10
MSE da rodada 2: 7.0143

Rodada 3/10
MSE da rodada 3: 8.7981

Rodada 4/10
MSE da rodada 4: 8.9483

Rodada 5/10


[I 2025-05-04 18:56:13,953] Trial 0 finished with value: 3.674291035840617 and parameters: {}. Best is trial 0 with value: 3.674291035840617.


MSE da rodada 5: 8.6841

[I 2025-05-04 18:56:13,988] A new study created in memory with name: no-name-ce133ac0-ea62-41a9-bdd9-d8017061a177
[I 2025-05-04 18:56:14,001] Trial 0 finished with value: 9.617530410171687 and parameters: {}. Best is trial 0 with value: 9.617530410171687.
[I 2025-05-04 18:56:14,023] A new study created in memory with name: no-name-27702dff-037c-4ff2-9d7e-0346cf080df5
[I 2025-05-04 18:56:14,029] Trial 0 finished with value: 6.760213950960017 and parameters: {}. Best is trial 0 with value: 6.760213950960017.
[I 2025-05-04 18:56:14,045] A new study created in memory with name: no-name-49f602e2-3be6-414d-a742-c1936f9d7840
[I 2025-05-04 18:56:14,062] Trial 0 finished with value: 9.808342232992597 and parameters: {}. Best is trial 0 with value: 9.808342232992597.
[I 2025-05-04 18:56:14,076] A new study created in memory with name: no-name-54f38345-ab90-4fe2-b1ee-d8827668cd07
[I 2025-05-04 18:56:14,092] Trial 0 finished with value: 5.855449266203709 and parameters: {}. Best is trial 0 with va



Rodada 6/10
MSE da rodada 6: 6.3068

Rodada 7/10
MSE da rodada 7: 5.8110

Rodada 8/10
MSE da rodada 8: 5.7530

Rodada 9/10
MSE da rodada 9: 7.3891

Rodada 10/10
MSE da rodada 10: 8.5238

Resumo após 10 execuções:
Média dos MSEs: 7.5433
Desvio padrão dos MSEs: 1.1953
Melhor MSE individual: 5.7530
Melhor modelo salvo em: Modelos\best_linear_regressor_model_optuna.pkl


## XGBoost

Resultados:

GridSearch:
* Média dos MSEs: 3.3815
* Desvio padrão dos MSEs: 0.6465
* Melhor MSE individual: 2.5832
* Tempo: 612m 57s

Optuna:
* Média dos MSEs: 3.3109
* Desvio padrão dos MSEs: 0.5015
* Melhor MSE individual: 2.5832
* Tempo: 36m 52s


GridSearch

In [ ]:
# Definindo os hiperparâmetros para o GridSearch
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400, 500,600,700,800,900,1000],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 1.0],  # mantido apenas uma vez
    'max_depth': [3, 5, 7, 9, 11],
}

# Inicializando o modelo Random Forest
xgb_reg = LinearRegression() # Regressor logistico
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Configurando o GridSearchCV com PredefinedSplit
grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Métrica de avaliação
    cv=ps,  # Número de folds para validação cruzada
    verbose=1,  # Mostra o progresso do tuning
    n_jobs=1  # Usa todos os núcleos disponíveis
)

# Treinando o GridSearchCV
grid_search.fit(X_full, y_full)

# Obtendo o melhor modelo

best_xgb_reg = grid_search.best_estimator_

# Avaliando o modelo nos dados de teste
predictions = best_xgb_reg.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (mse) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_xgb_model_grid.pkl')
joblib.dump(best_xgb_reg, model_path)
print(f"Modelo salvo em: {model_path}")

Fitting 1 folds for each of 5280 candidates, totalling 5280 fits
Mean Squared Error (mse) on test data: 2.795087678116888
Modelo salvo em: Modelos\best_xgb_model_grid.pkl


In [27]:
# Hiperparâmetros para o GridSearch
param_grid = {
    'n_estimators': [ 500, 600, 700, 800, 900, 1000],
    'learning_rate': [0.001, 0.01],
    'gamma': [0.1, 0.2, 0.3],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 1.0],
    'max_depth': [3, 5, 7, 9],
}

mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=40+i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=40+i
    )

    # Dados de treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    sy_30_y_val = y_val
    sy_30_y_test = y_test

    # Reconstruindo DataFrames escalados
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=sy_x_train.columns, index=X_test.index)

    # Junção de treino + validação
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, sy_30_y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Grid search
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

    grid_search = GridSearchCV(
        estimator=xgb_reg,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=ps,
        verbose=0,
        n_jobs=1
    )
    grid_search.fit(X_full, y_full)

    # Re-treinar modelo com melhores hiperparâmetros, agora só com treino (sem val)
    best_params = grid_search.best_params_
    retrain_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **best_params)
    retrain_model.fit(X_full, y_full)

    # Avaliação final
    predictions = retrain_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(sy_30_y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = retrain_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo da melhor rodada
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_xgb_model_grid.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")


Rodada 1/10
MSE da rodada 1: 3.1481

Rodada 2/10
MSE da rodada 2: 4.2340

Rodada 3/10
MSE da rodada 3: 2.7208

Rodada 4/10
MSE da rodada 4: 4.7078

Rodada 5/10
MSE da rodada 5: 2.6961

Rodada 6/10
MSE da rodada 6: 3.5697

Rodada 7/10
MSE da rodada 7: 3.5791

Rodada 8/10
MSE da rodada 8: 3.3294

Rodada 9/10
MSE da rodada 9: 2.5832

Rodada 10/10
MSE da rodada 10: 3.2472

Resumo após 10 execuções:
Média dos MSEs: 3.3815
Desvio padrão dos MSEs: 0.6465
Melhor MSE individual: 2.5832
Melhor modelo salvo em: Modelos\best_xgb_model_grid.pkl


Optuna

In [30]:
# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)


# Função objetivo para o Optuna
def objective(trial):
    # Sugerindo os hiperparâmetros
    n_estimators = trial.suggest_int("n_estimators", 50, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 3, 10, step=1)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.3, step=0.0005)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    gamma = trial.suggest_float("gamma", 0, 0.5, step=0.1)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 10, step=1)
    
    

    # Criando o modelo com os hiperparâmetros sugeridos
    xgb_reg = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        objective='reg:squarederror',
        gamma=gamma,
        min_child_weight=min_child_weight,
        
        random_state=42
    )

    # Treinando o modelo nos dados de treino
    xgb_reg.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])

    # Avaliando o modelo nos dados de validação
    predictions = xgb_reg.predict(X_full[ps.test_fold == 0])
    mse = mean_squared_error(y_full[ps.test_fold == 0], predictions)

    return mse  # O objetivo é minimizar o MSE

# Criando o estudo do Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Número de tentativas

# Obtendo os melhores hiperparâmetros
best_params = study.best_params
print("Melhores hiperparâmetros:", best_params)


# Treinando o modelo final com os melhores hiperparâmetros
best_xgb = xgb.XGBRegressor(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    learning_rate=best_params["learning_rate"],
    subsample=best_params["subsample"],
    gamma=best_params["gamma"],
    min_child_weight=best_params["min_child_weight"],
    objective='reg:squarederror',
    random_state=42
)
best_xgb.fit(sy_x_train_scaled, sy_y_train)

# Avaliando o modelo nos dados de teste
predictions = best_xgb.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_xgb_model_optuna.pkl')
joblib.dump(best_xgb, model_path)
print(f"Modelo salvo em: {model_path}")

[I 2025-04-24 14:19:00,280] A new study created in memory with name: no-name-d0c9ec92-0047-421f-aae5-462d1c780571
[I 2025-04-24 14:19:01,015] Trial 0 finished with value: 2.9025841908281023 and parameters: {'n_estimators': 250, 'max_depth': 7, 'learning_rate': 0.0485, 'subsample': 0.8, 'gamma': 0.1, 'min_child_weight': 7}. Best is trial 0 with value: 2.9025841908281023.
[I 2025-04-24 14:19:01,633] Trial 1 finished with value: 3.101864530634946 and parameters: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.10250000000000001, 'subsample': 0.9, 'gamma': 0.5, 'min_child_weight': 9}. Best is trial 0 with value: 2.9025841908281023.
[I 2025-04-24 14:19:01,911] Trial 2 finished with value: 3.3768841974123456 and parameters: {'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.1, 'subsample': 0.9, 'gamma': 0.0, 'min_child_weight': 1}. Best is trial 0 with value: 2.9025841908281023.
[I 2025-04-24 14:19:02,453] Trial 3 finished with value: 3.1724126058006505 and parameters: {'n_estim

Melhores hiperparâmetros: {'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.127, 'subsample': 0.6, 'gamma': 0.2, 'min_child_weight': 8}
Mean Squared Error (MSE) on test data: 2.8858393687086332
Modelo salvo em: Modelos\best_xgb_model_optuna.pkl


In [29]:
mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=40+i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=40+i
    )

    # Concatenando dados externos para o treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    # Reconstruindo DataFrames
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=sy_x_train.columns, index=X_test.index)

    # Concatenação treino + val para o fit
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Função objetivo do Optuna
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int("n_estimators", 500, 1000, step=100),
            'max_depth': trial.suggest_int("max_depth", 3, 9),
            'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01, step=0.001),
            'subsample': trial.suggest_float("subsample", 0.8, 1.0, step=0.1),
            'gamma': trial.suggest_float("gamma", 0.1, 0.3, step=0.1),
            'min_child_weight': trial.suggest_int("min_child_weight", 1, 3),
            'objective': 'reg:squarederror',
            'random_state': 42,
        }

        model = xgb.XGBRegressor(**params)
        model.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])
        preds = model.predict(X_full[ps.test_fold == 0])
        mse = mean_squared_error(y_full[ps.test_fold == 0], preds)
        return mse

    # Rodando o estudo
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, show_progress_bar=False)

    best_params = study.best_params
    print("Melhores hiperparâmetros:", best_params)

    # Treina modelo final com treino + val
    final_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **best_params)
    final_model.fit(X_full, y_full)

    # Avaliação em teste
    predictions = final_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = final_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_xgb_model_optuna.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")

[I 2025-05-04 07:19:45,372] A new study created in memory with name: no-name-f9772409-3488-470b-bd8d-4cec9d85c349



Rodada 1/10


[I 2025-05-04 07:19:52,367] Trial 0 finished with value: 4.527701345637087 and parameters: {'n_estimators': 800, 'max_depth': 8, 'learning_rate': 0.007, 'subsample': 0.9, 'gamma': 0.3, 'min_child_weight': 3}. Best is trial 0 with value: 4.527701345637087.
[I 2025-05-04 07:19:54,390] Trial 1 finished with value: 4.262052607185831 and parameters: {'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.005, 'subsample': 0.8, 'gamma': 0.3, 'min_child_weight': 2}. Best is trial 1 with value: 4.262052607185831.
[I 2025-05-04 07:19:55,945] Trial 2 finished with value: 4.371445556670445 and parameters: {'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.006, 'subsample': 0.8, 'gamma': 0.2, 'min_child_weight': 1}. Best is trial 1 with value: 4.262052607185831.
[I 2025-05-04 07:20:06,521] Trial 3 finished with value: 5.5922038580128355 and parameters: {'n_estimators': 800, 'max_depth': 9, 'learning_rate': 0.001, 'subsample': 1.0, 'gamma': 0.2, 'min_child_weight': 3}. Best is trial 1 with v

Melhores hiperparâmetros: {'n_estimators': 700, 'max_depth': 3, 'learning_rate': 0.006, 'subsample': 1.0, 'gamma': 0.1, 'min_child_weight': 2}


[I 2025-05-04 07:21:56,789] A new study created in memory with name: no-name-54ac5349-338b-4e8a-a7b0-ce460e501140


...

Melhores hiperparâmetros: {'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 1.0, 'gamma': 0.1, 'min_child_weight': 3}


[I 2025-05-04 07:54:22,949] A new study created in memory with name: no-name-a2098b73-92b3-4ac5-9455-db6f65e7a08a


MSE da rodada 9: 2.5832

Rodada 10/10


[I 2025-05-04 07:54:26,473] Trial 0 finished with value: 3.3458750129829475 and parameters: {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.002, 'subsample': 0.8, 'gamma': 0.2, 'min_child_weight': 1}. Best is trial 0 with value: 3.3458750129829475.
[I 2025-05-04 07:54:39,476] Trial 1 finished with value: 4.133456921576004 and parameters: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.002, 'subsample': 1.0, 'gamma': 0.1, 'min_child_weight': 1}. Best is trial 0 with value: 3.3458750129829475.
[I 2025-05-04 07:54:42,582] Trial 2 finished with value: 5.250167575073808 and parameters: {'n_estimators': 600, 'max_depth': 6, 'learning_rate': 0.001, 'subsample': 0.9, 'gamma': 0.1, 'min_child_weight': 1}. Best is trial 0 with value: 3.3458750129829475.
[I 2025-05-04 07:54:49,155] Trial 3 finished with value: 3.455293439156808 and parameters: {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.003, 'subsample': 1.0, 'gamma': 0.2, 'min_child_weight': 2}. Best is trial 0 w

Melhores hiperparâmetros: {'n_estimators': 900, 'max_depth': 4, 'learning_rate': 0.008, 'subsample': 1.0, 'gamma': 0.3, 'min_child_weight': 3}
MSE da rodada 10: 3.0429

Resumo após 10 execuções:
Média dos MSEs: 3.3109
Desvio padrão dos MSEs: 0.5015
Melhor MSE individual: 2.5832
Melhor modelo salvo em: Modelos\best_xgb_model_optuna.pkl


## MLP

Resultados:

GridSearch:
* Média dos MSEs: 4.6351
* Desvio padrão dos MSEs: 0.6425
* Melhor MSE individual: 
* Tempo: 220m 3s

Optuna:
* Média dos MSEs: 4.6210
* Desvio padrão dos MSEs: 0.8232
* Melhor MSE individual: 3.2475
* Tempo: 364m 16s


GridSearch

In [25]:


# Definindo os hiperparâmetros para o GridSearch
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100),(50, 50, 50), (100, 50, 50), (100, 100, 50), (100, 100, 100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001, 0.01]
}

# Inicializando o modelo MLP
mlp_reg = MLPRegressor(random_state=42, max_iter=500)

# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Configurando o GridSearchCV com PredefinedSplit
grid_search = GridSearchCV(
    estimator=mlp_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Métrica de avaliação
    cv=ps,  # Número de folds para validação cruzada
    verbose=3,  # Mostra o progresso do tuning
    n_jobs=-1  # Usa todos os núcleos disponíveis
)

# Treinando o GridSearchCV
grid_search.fit(X_full, y_full)

# Obtendo o melhor modelo
best_mlp_reg = grid_search.best_estimator_

# Avaliando o modelo nos dados de teste
predictions = best_mlp_reg.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_mlp_model_grid.pkl')
joblib.dump(best_mlp_reg, model_path)
print(f"Modelo salvo em: {model_path}")

Fitting 1 folds for each of 216 candidates, totalling 216 fits
Mean Squared Error (MSE) on test data: 3.3342401641444734
Modelo salvo em: Modelos\best_mlp_model_grid.pkl


c:\faculdade\TCC_Music\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


In [28]:
# Definindo os hiperparâmetros para o GridSearch
param_grid = {
    'hidden_layer_sizes': [ (100, 100), (50, 50, 50), (100, 50, 50), (100, 100, 50), (100, 100, 100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001]
}

# Variáveis para armazenar os melhores modelos e MSE
best_mlp_model = None
best_mse = float('inf')
mse_list = []

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Dividindo os dados de até 30 dias em treino/validação/teste
    X_train_val, X_test, y_train_val, y_test = train_test_split(youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=40+i)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.20, random_state=40+i)

    # Concatenação com os demais dados
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler_all = MinMaxScaler()
    sy_x_train_scaled = scaler_all.fit_transform(sy_x_train)
    sy_x_val_scaled = scaler_all.transform(X_val)
    sy_x_test_scaled = scaler_all.transform(X_test)

    # Convertendo de volta para DataFrames
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_x_val_scaled_df = pd.DataFrame(sy_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_x_test_scaled_df = pd.DataFrame(sy_x_test_scaled, columns=sy_x_train.columns, index=X_test.index)

    # Concatena treino e validação
    X_full = pd.concat([sy_x_train_scaled_df, sy_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # Cria PredefinedSplit
    validation_fold = np.concatenate([np.full(len(sy_x_train), -1), np.zeros(len(X_val))])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Inicializa o modelo base
    mlp_reg = MLPRegressor(random_state=i, max_iter=1000)

    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=mlp_reg,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=ps,
        verbose=0,
        n_jobs=-1
    )

    # Treinamento
    grid_search.fit(X_full, y_full)

    # Obtém os melhores hiperparâmetros
    best_params = grid_search.best_params_

    # Reentreina o modelo com os melhores hiperparâmetros usando treino + validação
    best_mlp = MLPRegressor(**best_params, random_state=42, max_iter=1000)
    best_mlp.fit(X_full, y_full)

    # Avaliação no conjunto de teste
    predictions = best_mlp.predict(sy_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    # Verifica se é o melhor MSE
    if mse < best_mse:
        best_mse = mse
        best_mlp_model = best_mlp

# Resumo final
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_mlp_model_grid.pkl')
joblib.dump(best_mlp_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")


Rodada 1/10
MSE da rodada 1: 5.1824

Rodada 2/10
MSE da rodada 2: 5.2343

Rodada 3/10
MSE da rodada 3: 3.2215

Rodada 4/10
MSE da rodada 4: 4.8412

Rodada 5/10
MSE da rodada 5: 4.2848

Rodada 6/10
MSE da rodada 6: 4.6863

Rodada 7/10
MSE da rodada 7: 4.4775

Rodada 8/10
MSE da rodada 8: 4.7943

Rodada 9/10
MSE da rodada 9: 4.0432

Rodada 10/10
MSE da rodada 10: 5.5854

Resumo após 10 execuções:
Média dos MSEs: 4.6351
Desvio padrão dos MSEs: 0.6425
Melhor modelo salvo em: Modelos\best_mlp_model_grid.pkl


Optuna

In [23]:
# Converte os arrays normalizados de volta para DataFrame
sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=sy_30_x_val.columns, index=sy_30_x_val.index)

# Junta treino e validação
X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
y_full = pd.concat([sy_y_train, sy_30_y_val])

# Cria vetor de validação: -1 para treino, 0 para validação
validation_fold = np.concatenate([
    np.full(len(sy_x_train), -1),
    np.zeros(len(sy_30_x_val))
])

# Cria o PredefinedSplit
ps = PredefinedSplit(test_fold=validation_fold)

# Função objetivo para o Optuna
def objective(trial):
    # Sugerindo os hiperparâmetros
     # Escolhe número de camadas: 1, 2 ou 3
    n_layers = trial.suggest_int('n_layers', 1, 3)
    
    # Para cada camada, escolher número de neurônios
    hidden_layer_sizes = []
    for i in range(n_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 50, 200, step=10)
        hidden_layer_sizes.append(num_units)
    
    hidden_layer_sizes = tuple(hidden_layer_sizes)  # vira tupla, como o MLP espera
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])
    alpha = trial.suggest_float("alpha", 0.0001, 0.01, log=True)
    learning_rate = trial.suggest_categorical("learning_rate", ["constant", "adaptive"])

    # Criando o modelo com os hiperparâmetros sugeridos
    mlp_reg = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=1000,
        random_state=42
    )

    # Treinando o modelo nos dados de treino
    mlp_reg.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])

    # Avaliando o modelo nos dados de validação
    predictions = mlp_reg.predict(X_full[ps.test_fold == 0])
    mse = mean_squared_error(y_full[ps.test_fold == 0], predictions)

    return mse  # O objetivo é minimizar o MSE

# Criando o estudo do Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Número de tentativas

# Obtendo os melhores hiperparâmetros
best_params = study.best_params

# Construindo o hidden_layer_sizes a partir dos melhores parâmetros
hidden_layer_sizes = []
for i in range(best_params["n_layers"]):
    hidden_layer_sizes.append(best_params[f"n_units_layer_{i}"])
best_params["hidden_layer_sizes"] = tuple(hidden_layer_sizes)

print("Melhores hiperparâmetros:", best_params)

# Treinando o modelo final com os melhores hiperparâmetros
best_mlp = MLPRegressor(
    hidden_layer_sizes=best_params["hidden_layer_sizes"],
    activation=best_params["activation"],
    solver=best_params["solver"],
    alpha=best_params["alpha"],
    learning_rate=best_params["learning_rate"],
    max_iter=1000,
    random_state=42
)
best_mlp.fit(sy_x_train_scaled, sy_y_train)

# Avaliando o modelo nos dados de teste
predictions = best_mlp.predict(sy_30_x_test_scaled)
mse = mean_squared_error(sy_30_y_test, predictions)
print(f"Mean Squared Error (MSE) on test data: {mse}")

# Salvando o modelo na pasta 'Modelos'
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_mlp_model_optuna.pkl')
joblib.dump(best_mlp, model_path)
print(f"Modelo salvo em: {model_path}")

[I 2025-04-28 09:49:18,683] A new study created in memory with name: no-name-593c19ba-4856-4ffc-84a5-7ada84bd71bc
[I 2025-04-28 09:49:53,402] Trial 0 finished with value: 4.253595743081349 and parameters: {'n_layers': 1, 'n_units_layer_0': 70, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.006071323986349375, 'learning_rate': 'constant'}. Best is trial 0 with value: 4.253595743081349.
[I 2025-04-28 09:54:09,362] Trial 1 finished with value: 4.783693931466111 and parameters: {'n_layers': 2, 'n_units_layer_0': 180, 'n_units_layer_1': 190, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.00045592803267607867, 'learning_rate': 'constant'}. Best is trial 0 with value: 4.253595743081349.
[I 2025-04-28 09:54:51,381] Trial 2 finished with value: 4.579312869609497 and parameters: {'n_layers': 3, 'n_units_layer_0': 110, 'n_units_layer_1': 80, 'n_units_layer_2': 100, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.002598256618167818, 'learning_rate': 'constant'}. Best is trial 0 with val

Melhores hiperparâmetros: {'n_layers': 1, 'n_units_layer_0': 140, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.00020915691387984604, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (140,)}
Mean Squared Error (MSE) on test data: 3.092867299798431
Modelo salvo em: Modelos\best_mlp_model_optuna.pkl


In [30]:
mse_list = []
best_overall_model = None
lowest_mse = float('inf')

for i in range(10):
    print(f"\nRodada {i+1}/10")

    # Divisão dos dados
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=40+i
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.20, random_state=40+i
    )

    # Concatena dados externos com treino
    sy_x_train = pd.concat([X_train, spotify_youtube_df_31_90X_train, spotify_youtube_df_91_365X_train, spotify_youtube_df_366_more_x])
    sy_y_train = pd.concat([y_train, spotify_youtube_df_31_90y_train, spotify_youtube_df_91_365y_train, spotify_youtube_df_366_more_y])

    # Normalização
    scaler = MinMaxScaler()
    sy_x_train_scaled = scaler.fit_transform(sy_x_train)
    sy_30_x_val_scaled = scaler.transform(X_val)
    sy_30_x_test_scaled = scaler.transform(X_test)

    # Reconstruindo DataFrames
    sy_x_train_scaled_df = pd.DataFrame(sy_x_train_scaled, columns=sy_x_train.columns, index=sy_x_train.index)
    sy_30_x_val_scaled_df = pd.DataFrame(sy_30_x_val_scaled, columns=X_val.columns, index=X_val.index)
    sy_30_x_test_scaled_df = pd.DataFrame(sy_30_x_test_scaled, columns=X_test.columns, index=X_test.index)

    # Junta treino + val
    X_full = pd.concat([sy_x_train_scaled_df, sy_30_x_val_scaled_df])
    y_full = pd.concat([sy_y_train, y_val])

    # PredefinedSplit
    validation_fold = np.concatenate([
        np.full(len(sy_x_train), -1),
        np.zeros(len(X_val))
    ])
    ps = PredefinedSplit(test_fold=validation_fold)

    # Função objetivo do Optuna
    def objective(trial):
        # Camadas ocultas
        n_layers = trial.suggest_int('n_layers', 1, 3)
        hidden_layer_sizes = tuple(
            trial.suggest_int(f'n_units_layer_{i}', 50, 200, step=10) for i in range(n_layers)
        )

        # Outros hiperparâmetros
        activation = trial.suggest_categorical("activation", ["relu", "tanh"])
        solver = trial.suggest_categorical("solver", ["adam", "sgd"])
        alpha = trial.suggest_float("alpha", 0.0001, 0.01, log=True)
        learning_rate = trial.suggest_categorical("learning_rate", ["constant", "adaptive"])

        model = MLPRegressor(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            alpha=alpha,
            learning_rate=learning_rate,
            max_iter=1000,
            random_state=42
        )

        model.fit(X_full[ps.test_fold == -1], y_full[ps.test_fold == -1])
        preds = model.predict(X_full[ps.test_fold == 0])
        mse = mean_squared_error(y_full[ps.test_fold == 0], preds)
        return mse

    # Rodando Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, show_progress_bar=False)

    best_params = study.best_params
    hidden_layer_sizes = tuple(
        best_params[f"n_units_layer_{i}"] for i in range(best_params["n_layers"])
    )
    best_params["hidden_layer_sizes"] = hidden_layer_sizes
    print("Melhores hiperparâmetros:", best_params)

    # Treinando o modelo final com melhores parâmetros
    final_model = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=best_params["activation"],
        solver=best_params["solver"],
        alpha=best_params["alpha"],
        learning_rate=best_params["learning_rate"],
        max_iter=1000,
        random_state=42
    )
    final_model.fit(X_full, y_full)

    # Avaliação em teste
    predictions = final_model.predict(sy_30_x_test_scaled_df)
    mse = mean_squared_error(y_test, predictions)
    mse_list.append(mse)
    print(f"MSE da rodada {i+1}: {mse:.4f}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_overall_model = final_model

# Estatísticas finais
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print("\nResumo após 10 execuções:")
print(f"Média dos MSEs: {mean_mse:.4f}")
print(f"Desvio padrão dos MSEs: {std_mse:.4f}")
print(f"Melhor MSE individual: {lowest_mse:.4f}")

# Salvando o melhor modelo
os.makedirs('Modelos', exist_ok=True)
model_path = os.path.join('Modelos', 'best_mlp_model_optuna.pkl')
joblib.dump(best_overall_model, model_path)
print(f"Melhor modelo salvo em: {model_path}")

[I 2025-05-04 07:56:37,861] A new study created in memory with name: no-name-adc330ee-111b-4320-ac79-4e9f439fd7c8



Rodada 1/10


[I 2025-05-04 07:56:59,510] Trial 0 finished with value: 7.192820279139715 and parameters: {'n_layers': 1, 'n_units_layer_0': 140, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.006090078102921381, 'learning_rate': 'constant'}. Best is trial 0 with value: 7.192820279139715.
[I 2025-05-04 07:57:16,171] Trial 1 finished with value: 5.7246338440041065 and parameters: {'n_layers': 1, 'n_units_layer_0': 160, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.00010981506553327514, 'learning_rate': 'adaptive'}. Best is trial 1 with value: 5.7246338440041065.
[I 2025-05-04 07:58:19,021] Trial 2 finished with value: 7.5692151973233806 and parameters: {'n_layers': 3, 'n_units_layer_0': 80, 'n_units_layer_1': 90, 'n_units_layer_2': 80, 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0008410825691513429, 'learning_rate': 'adaptive'}. Best is trial 1 with value: 5.7246338440041065.
[I 2025-05-04 08:00:49,370] Trial 3 finished with value: 7.382959734581774 and parameters: {'n_layers': 3, 'n_un

Melhores hiperparâmetros: {'n_layers': 2, 'n_units_layer_0': 60, 'n_units_layer_1': 80, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0043277392364747405, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (60, 80)}


[I 2025-05-04 08:46:19,598] A new study created in memory with name: no-name-0147bb7a-4353-4346-8b39-981628b6e3e5


...

Melhores hiperparâmetros: {'n_layers': 1, 'n_units_layer_0': 80, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.00011253880897789654, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (80,)}


[I 2025-05-04 13:33:10,322] A new study created in memory with name: no-name-f696fd14-a260-4c70-8958-ce929f9fc7d5


MSE da rodada 9: 4.4158

Rodada 10/10


[I 2025-05-04 13:33:27,872] Trial 0 finished with value: 3.9032380374637667 and parameters: {'n_layers': 2, 'n_units_layer_0': 130, 'n_units_layer_1': 160, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.004721928905799051, 'learning_rate': 'adaptive'}. Best is trial 0 with value: 3.9032380374637667.
[I 2025-05-04 13:34:31,360] Trial 1 finished with value: 6.474269292727304 and parameters: {'n_layers': 3, 'n_units_layer_0': 170, 'n_units_layer_1': 70, 'n_units_layer_2': 190, 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.007820991032179469, 'learning_rate': 'constant'}. Best is trial 0 with value: 3.9032380374637667.
[I 2025-05-04 13:34:44,654] Trial 2 finished with value: 6.7349515423898945 and parameters: {'n_layers': 1, 'n_units_layer_0': 150, 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0013268918901219139, 'learning_rate': 'adaptive'}. Best is trial 0 with value: 3.9032380374637667.
[I 2025-05-04 13:34:48,716] Trial 3 finished with value: 7.686213602780017 and parameter

Melhores hiperparâmetros: {'n_layers': 1, 'n_units_layer_0': 180, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0010966978409251748, 'learning_rate': 'constant', 'hidden_layer_sizes': (180,)}
MSE da rodada 10: 4.0841

Resumo após 10 execuções:
Média dos MSEs: 4.6210
Desvio padrão dos MSEs: 0.8232
Melhor MSE individual: 3.2475
Melhor modelo salvo em: Modelos\best_mlp_model_optuna.pkl
